# Machine Learning with N-Gram Language Models

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JamesMTucker/DATA_340_NLP/blob/master/Fall_2023/notebooks/07_NGram_Models.ipynb)

Lecture outline:

* Missed Lecture 9 Material
* Assignment Updates and Clarifications
* Project Updates, Clarifications, and Suggestions (Twitter scraper)
* N-Gram Language Models
  * Some background ideas and assumptions
  * N-Gram Character Models
  * N-Gram Word Models

## N-Gram Language Models

### Some background ideas and assumptions

Please turn in your _________ 
- [ ] notebook?
- [ ] pdf?
- [ ] html?
- [ ] homework?
- [ ] report?
- [ ] slides?
- [ ] poster?
- [ ] video?

- "Models that assign probabilities to sequences of words are ... language models." (Jurafsky & Martin, p. 32)
- Deep neural networks Language Models often assoctiate the training task of masked language modeling (MLM) with the task of predicting the next word in a sequence. Example: [BiomedNLP](https://huggingface.co/microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext?text=%5BMASK%5D+is+a+tumor+suppressor+gene)

### Some important concepts:

- Smoothing
- Backoff
- Perplexity


## N-Grams

$P(w|h)$ is the probability of a word $w$ given a history $h$.

### Computing the probability of a sequence

$$P(X_1, X_2, \dots, X_n) = P(X_1)P(X_2|X_1)P(X_3|X_1, X_2) \dots P(X_n|X_1, X_2, \dots, X_{n-1})$$

$$=\sum_{k=1}^{n} P(X_k|X_1, X_2, \dots, X_{k-1})$$

Approximate the history by using the hidden markov assumption:

$$P(w_n|w_{1:n-1}) \approx P(w_n|w_{n-1})$$



Let $N$ be the size of gram model. We can approximate the probability of a sequence by using the following equation:

$$P(w_{1:n}) \approx \sum^{n}_{k=1} P(w_k|w_{k-1})$$

### Maximum Likelihood Estimation

$$P(w_n|w_{n-1}) = \frac{C(w_{n-1}, w_n)}{\sum_w C(w_{n-1}, w)}$$

"We can simplify this equation, since the sum of all bigram counts that start with a given word $w_{n-1}$ must be equal to the unigram count for that word $w_{n-1}$..." (Jurafsky & Martin, p. 34)

Explain why this is so.

Thus, we can simplify the equation to:

$$P(w_n|w_{n-1}) = \frac{C(w_{n-1}, w_n)}{C(w_{n-1})}$$

## Example 1: Green Eggs and Ham

What does this all mean? Let's code it up and seek for a better understanding.


Let's use the following corpus:


In [ ]:
corpus = "I am Sam. Sam I am. I do not like green eggs and ham."

sentences = corpus.split(".")
sentences = [f"<s> {s.strip()} </s>" for s in sentences if s.strip() != ""]
sentences

In [ ]:
## Create bigram tokens for each sentence
tokens = [s.split() for s in sentences]
bigrams = [bigram for sentence in tokens for bigram in zip(sentence[:-1], sentence[1:])]

import nltk
from collections import Counter

# flatten the list of tokens
tokens = [token for sentence in tokens for token in sentence]

# create a frequency distribution
unigram_freq = {k: v for k, v in sorted(Counter(tokens).items(), key=lambda item: item[1], reverse=True)}

## Generate probability table using the relative frequency
freq = nltk.FreqDist(bigrams)

In [ ]:
freq

In [ ]:
## Compute the probability of each bigram
probs = {k: v / unigram_freq[k[0]] for k, v in freq.items()}
probs

In [ ]:
## Generate a heatmap of the probabilities
import pandas as pd
import numpy as np
# Generate a matrix of the probabilities with the bigrams as the index
df = pd.DataFrame.from_dict(probs, orient="index", columns=["Probability"])
df[['token_1', 'token_2']] = pd.DataFrame(df.index.tolist(), index=df.index)

In [ ]:
# Pivot the table to get the heatmap
df.pivot("token_1", "token_2", "Probability").replace(np.nan, f'{0:.1f}')

In [ ]:
## Generate a heatmap of the probabilities
import seaborn as sns

# Generate heatmap of the bigram probabilities
sns.heatmap(df.pivot("token_1", "token_2", "Probability"), annot=True, fmt=".2f")

## Example 2: A Character Bigram Language Model

In this example, we will create a character bigram language model. We will use the _Lord of the Rings_ as our corpus.

### Step 1: Read the text

In [ ]:
## Import Lord of the rings text

# use google to load the data from drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

datasets_dir = "/content/drive/My Drive/DATA_340_3_NLP/Datasets/"

# get the txt files
filenames = [os.path.join(datasets_dir, f) for f in os.listdir(datasets_dir) if f.endswith(".txt") and 'LOTR' in f]

# read the files
corpus = []

# read 
for f in filenames:
    with open(f, 'r', encoding='ISO-8859-1') as file:
        corpus.append(file.read())
        
# remove whitespace from the corpus
corpus = " ".join(corpus)
corpus = " ".join(corpus.split())

### Step 2: Clean the text

In [ ]:
try:
  import unidecode
except ModuleNotFoundError:
  !pip install unidecode

In [ ]:
from unidecode import unidecode

text = unidecode(corpus)

### Step 3: Create the bigram model

In [ ]:
words = text.split(" ")

In [ ]:
## Create a bigram dictionary
bigrams = {}

# Since we need to mark the start and end of the sentence, we add a special character
for w in words:
    chs = ['_'] + list(w) + ['_']
    for ch1, ch2 in zip(chs[:-1], chs[1:]):
        bigrams[(ch1, ch2)] = bigrams.get((ch1, ch2), 0) + 1

In [ ]:
# Examine the bigrams
sorted(bigrams.items(), key=lambda kv: kv[1], reverse=True)[:10]

### Step 4: Create Encoder and Decoder for Characters

In [ ]:
# Create a list of characters from our data
chars = sorted(list(set(''.join(words))))

# Create a encoding dictionary
stoi = {c:i+1 for i, c in enumerate(chars)}
stoi['_'] = 0

# Create a decoding dictionary
itos = {i:c for c, i in stoi.items()}


In [ ]:
# examine the int to string dictionary
itos

### Step 5: Create a bigram frequency count matrix

In [ ]:
# We can use the Torch library to create our bigram matrix
import torch

# Since we our input is len(chars) x len(chars), we create a matrix of zeros
N = torch.zeros((len(chars)+1, len(chars)+1), dtype=torch.int32)

In [ ]:
for w in words:
    chs = ['_'] + list(w) + ['_']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        N[ix1, ix2] += 1

In [ ]:
# We can now see the counts of each bigram in our tensor
N[0, :]

### Step 6: Visualize the bigram count matrix

In [ ]:
## Let's visualize the bigram matrix
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(36,36))
plt.imshow(N, cmap='Blues')
for i in range(61):
    for j in range(61):
        chstr = itos[i] + itos[j]
        plt.text(j, i, chstr, ha='center', va='bottom', color='gray', fontsize=8)
        plt.text(j, i, N[i, j].item(), ha='center', va='top', color='gray', fontsize=8)
plt.axis('off');
plt.savefig('bigram_matrix.png');

### Step 7: Convert the bigram count matrix to a probability matrix

In [ ]:
## Sample from the probability distribution
p = N[0].float()

## normalize
p = p / p.sum()

## Sum to 1
p.sum()

### Step 8: Use our model to generate text

In [ ]:
## Create a probability matrix for all the bigrams

g = torch.Generator().manual_seed(2147483647)

## Our paragraph to write
paragraph = []

for i in range(1000):
    out = []
    ix = 0
    while True:
        p = N[ix].float()
        p = p / p.sum()
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix])
        if ix == 0:
            break
    paragraph.append(''.join(out))
    
paragraph = ''.join(paragraph)
paragraph = paragraph.replace('_', ' ')
paragraph

In [ ]:
## Did our model learn anything?

## Tolkein's style
for w in words[:10]:
    chs = ['_'] + list(w) + ['_']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        prob = N[ix1, ix2].item() / N[ix1].sum().item()
        print(f"{ch1} -> {ch2} : {prob:.4f}")

In [ ]:
## Our model's style

words = paragraph.split(" ")

for w in words[:10]:
    chs = ['_'] + list(w) + ['_']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        prob = N[ix1, ix2].item() / N[ix1].sum().item()
        print(f"{ch1} -> {ch2} : {prob:.4f}")


In [ ]:
## Step 9: Maximum Likelihood Estimation

In [ ]:
## Maximum Likelihood Estimation
import numpy as np

words = paragraph.split(" ")

## log(a*b) = log(a) + log(b)

for w in words[:10]:
    chs = ['_'] + list(w) + ['_']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        prob = N[ix1, ix2].item() / N[ix1].sum().item()
        logprob = np.log(prob)
        print(f"{ch1} -> {ch2} : {prob:.4f} {logprob:.4f}")

We want to a maximize the likelihood of the generated text with respect to the model parameters. (statistical inference)

[Log's are monotonic](https://www.wolframalpha.com/input?i=log%28x%29+from+0+to+1), so we can maximize the log-likelihood instead.

The smaller the log-likelihood, the better the model.


In [ ]:
import numpy as np

words = paragraph.split(" ")
log_likelihood = 0
n = 0


## log(a*b) = log(a) + log(b)

# for w in ['james']: # we can test any word we want
for w in ['jamez']: # a problem with this model is that it doesn't know how to spell
# for w in words[:10]:
    chs = ['_'] + list(w) + ['_']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        prob = N[ix1, ix2].item() / N[ix1].sum().item()
        logprob = np.log(prob)
        log_likelihood += logprob
        n += 1
        print(f"{ch1} -> {ch2} : {prob:.4f} {logprob:.4f}")

print(f'{log_likelihood=}')
nll = -log_likelihood
print(f'{nll=}')
print(f'{nll/n=}')

In [ ]:
import numpy as np

words = paragraph.split(" ")
log_likelihood = 0
n = 0


## log(a*b) = log(a) + log(b)

# for w in ['james']: # we can test any word we want
for w in ['jamez']: # a problem with this model is that it doesn't know how to spell
# for w in words[:10]:
    chs = ['_'] + list(w) + ['_']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        prob = N[ix1, ix2].item() / N[ix1].sum().item()
        logprob = np.log(prob)
        log_likelihood += logprob
        n += 1
        print(f"{ch1} -> {ch2} : {prob:.4f} {logprob:.4f}")

print(f'{log_likelihood=}')
nll = -log_likelihood
print(f'{nll=}')
print(f'{nll/n=}')

### Laplace Smoothing

In [ ]:
import numpy as np

words = paragraph.split(" ")
log_likelihood = 0
n = 0


## log(a*b) = log(a) + log(b)

# for w in ['james']: # we can test any word we want
for w in ['jamez']: # a problem with this model is that it doesn't know how to spell
# for w in words[:10]:
    chs = ['_'] + list(w) + ['_']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        prob = N[ix1, ix2].item() / N[ix1].sum().item() + 1 # add one for smoothing
        logprob = np.log(prob)
        log_likelihood += logprob
        n += 1
        print(f"{ch1} -> {ch2} : {prob:.4f} {logprob:.4f}")

print(f'{log_likelihood=}')
nll = -log_likelihood
print(f'{nll=}')
print(f'{nll/n=}')

## Our Trained Model


In [ ]:
g = torch.Generator().manual_seed(2345)

## Our paragraph to write
paragraph = []

for i in range(1000):
    out = []
    ix = 0
    while True:
        p = N[ix].float()
        p = p / p.sum()
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix])
        if ix == 0:
            break
    paragraph.append(''.join(out))
    
paragraph = ''.join(paragraph)
paragraph = paragraph.replace('_', ' ')
paragraph